In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import seaborn as sns
import shutil

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
import subprocess as sp
from sklearn.metrics import roc_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn import metrics
from sklearn.metrics import confusion_matrix

work_dir = "/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Publications/gene-regulatory-networks-in-epidermal-and-corneal-epithelia/"
os.chdir(f"{work_dir}/notebooks/Python_functions/")
from Python_scripts import distance_weight
from Python_scripts import make_bedfile_from_column
from Python_scripts import TSS_window_to_region
from Python_scripts import make_autopct
from Python_scripts import bedtool_closest

plt.style.use("classic")
%matplotlib inline
#%load_ext nb_black
#%reload_ext nb_black

In [3]:
class analysis:
    """contains all the first steps of my general analysis, including reading a filepath file, config file, and
    exporting those and the conda environment to a logdir.
    
    required input:
    datapaths_file, a tsv file containing all the other filepaths needed in the subsequent analysis
    config_file, a tsv file containing all type of settings and paramters
    output_dir, where to store the output generated by the analysis, it will generate 
    a log and figure dir in this ouput dir
    """

    def __init__(self, datapaths_file: str, config_file: str, output_dir: str, notebook_file: str):
        self.datapaths_file = datapaths_file
        self.config_file = config_file
        self.output_dir = output_dir
        self.files = pd.read_table(
            self.datapaths_file, sep="\t", comment="#", index_col=0, header = 0
        )
        self.config = pd.read_table(
            self.config_file, sep="\t", comment="#", index_col=0, header = 0
        )
        self.notebook_file = notebook_file
        self.logdir = f"{self.output_dir}/logs"
        self.figdir = f"{self.output_dir}/figures"
        self.trackhubdir = f"{self.output_dir}/trackhub"
        self.tmpdir = f"{self.output_dir}/tmp"
        Path(self.tmpdir).mkdir(parents=True, exist_ok=True)
        Path(self.logdir).mkdir(parents=True, exist_ok=True)
        Path(self.figdir).mkdir(parents=True, exist_ok=True)
        Path(self.trackhubdir).mkdir(parents=True, exist_ok=True)


    def save_settings(self, conda_path):
        """save the config and files to the logs, and make a copy of the conda env that his analysis
        is ran in. Takes a full path to the conda env location as input"""
        #save the conda environment
        !{conda_path} list > {self.logdir}/conda_env.txt
        # save both the files and settings of this run to the output_dir
        self.files.T.to_csv(f"{self.logdir}/filepaths.tsv", header=False, sep="\t")
        self.config.to_csv(f"{self.logdir}/config.tsv", header=False, sep="\t")


In [5]:
MotifEnr = analysis(
    datapaths_file=f"{work_dir}/data/Motif_enrichment_files/files.tsv",
    config_file=f"{work_dir}/data/Motif_enrichment_files/config.tsv",
    output_dir=f"{work_dir}/Motif_enrichment/02-05-2022",
    notebook_file=f'{work_dir}/notebooks/2_Motif_enrichment.ipynb',
)

In [6]:
TF_motif_file = MotifEnr.files.T.TF2motif[0]
TF_motif_file = pd.read_table(TF_motif_file, sep="\t", comment="#")
TF_motif_file["Factor"] = TF_motif_file["Factor"].str.upper()

In [7]:
# lets load the deseq2 expression data:
df_transcriptome = pd.read_table(
    MotifEnr.files.T.transcriptome[0],
    sep=";",
    index_col=0,
    header = 0,
)
df_transcriptome.columns = [str(col) + "_RNAseq" for col in df_transcriptome.columns]
df_transcriptome
conditions = [
    # (df_transcriptome.BaseMean_RNAseq < 5.0),
    (
        (df_transcriptome.padj_RNAseq < 0.01)
        & (df_transcriptome.log2FoldChange_RNAseq < -0.58)
    ),
    (
        (df_transcriptome.padj_RNAseq < 0.01)
        & (df_transcriptome.log2FoldChange_RNAseq > 0.58)
    ),
]
choices = ["LSC_high", "KC_high"]
df_transcriptome["prediction_type_RNAseq"] = np.select(
    conditions, choices, default="constant"
)

In [8]:
TF_motif_file = pd.read_table(MotifEnr.files.T.TF2motif[0], sep="\t", comment="#")
TF_motif_file["Factor"] = TF_motif_file["Factor"].str.upper()
genes_expressed = df_transcriptome[df_transcriptome.baseMean_RNAseq > 10]

# merge the motif database with expression data, if a TF is not measured its excluded (halfing the database in size)
expressed_TF_motif = TF_motif_file.merge(
    genes_expressed, left_on="Factor", right_index= True)

expressed_TF_motif.iloc[:, [0, 1, 2, 3]].to_csv(
    f"{MotifEnr.output_dir}/expressedTFs.motif2factors.txt",
    header=True,
    sep="\t",
    index=False,
)

print(
    f"ammount of TFs expressed and in motif db: {len(expressed_TF_motif.Factor.unique())}"
)

diff_TF_motif = expressed_TF_motif[
    expressed_TF_motif["prediction_type_RNAseq"] != "constant"
]
diff_TF_motif.iloc[:, [0, 1, 2, 3]].to_csv(
    f"{MotifEnr.output_dir}/diffTFs.motif2factors.txt",
    header=True,
    sep="\t",
    index=False,
)
print(
    f"ammount of TFs differentially expressed and in motif db: {len(diff_TF_motif.Factor.unique())}"
)


ammount of TFs expressed and in motif db: 502
ammount of TFs differentially expressed and in motif db: 85


In [22]:
# filter pfm database
pfm_file = open(MotifEnr.files.T.motif_pwm[0], "r")
all_motifs_dict = {}
pwm_list = []


new_motif = ""
for line in pfm_file:
    if line.startswith(">"):
        if new_motif != "":
            all_motifs_dict[new_motif] = pwm_list
            pwm_list = []
        new_motif = line.strip()
        new_motif = new_motif.strip(">")
    else:
        pwm_list.append([line.strip()])
pfm_file.close()

# only expressed TFs
expr_motifs_dict = {
    k: v for k, v in all_motifs_dict.items() if k in (list(expressed_TF_motif.Motif))
}
expr_pfm_file = open(
    f"{MotifEnr.output_dir}/expressedTFs.pfm",
    "w",
)
for TF, pwm in expr_motifs_dict.items():
    expr_pfm_file.write(f">{TF}\n")
    for line in pwm:
        expr_pfm_file.write(f'{"".join(line)}\n')
expr_pfm_file.close()

# only diff TFs
diff_motifs_dict = {
    k: v for k, v in all_motifs_dict.items() if k in (list(diff_TF_motif.Motif))
}
diff_pfm_file = open(
    f"{MotifEnr.output_dir}/diffTFs.pfm",
    "w",
)
for TF, pwm in diff_motifs_dict.items():
    diff_pfm_file.write(f">{TF}\n")
    for line in pwm:
        diff_pfm_file.write(f'{"".join(line)}\n')
diff_pfm_file.close()

#lets also copy a original to the same location as the filtered databases
shutil.copyfile(MotifEnr.files.T.motif_pwm[0], f"{MotifEnr.output_dir}/gimme.vertebrate.v5.0.pfm")
shutil.copyfile(MotifEnr.files.T.TF2motif[0], f"{MotifEnr.output_dir}/gimme.vertebrate.v5.0.motif2factors.txt")

'/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/Publications/gene-regulatory-networks-in-epidermal-and-corneal-epithelia//Motif_enrichment/02-05-2022/gimme.vertebrate.v5.0.motif2factors.txt'

###  after collapsing the TF-motif database, lets perform motif enrichment analysis on  all differential/highly variable regions!

In [10]:
region_data = MotifEnr.files.T.CREregion_data[0]
region_data = pd.read_table(region_data, sep=",", comment="#", header=0, index_col=0)
#lets filter for only the differential/highly variable regions:
annotation_data = region_data.loc[
    :, region_data.columns.str.contains("prediction_type")
]
# lets make a 200bp window
region_data["summit"] = region_data.index
region_data[["Chrom", "ChromStart", "ChromEnd"]] = region_data.summit.str.split(
    "[:-]", expand=True
)
region_data["ChromStart"] = region_data["ChromStart"].astype(int) - 100
region_data["ChromEnd"] = region_data["ChromEnd"].astype(int) + 100
region_data["200bpsummit"] = (
    region_data["Chrom"].astype(str)
    + ":"
    + region_data["ChromStart"].astype(str)
    + "-"
    + region_data["ChromEnd"].astype(str)
)
region_data.index = region_data["200bpsummit"]

In [11]:
annotation_data

,prediction_type_H3K27ac,prediction_type_H3K4me3,prediction_type_ATAC,prediction_type_H3K27me3
summit_H3K27ac,,,,
1:629078-629278,constant,low_signal,LSC_high,low_signal
1:629843-630043,constant,low_signal,constant,low_signal
1:630422-630622,constant,low_signal,LSC_high,constant
1:633977-634177,LSC_high,low_signal,constant,low_signal
1:778592-778792,constant,constant,constant,low_signal
...,...,...,...,...
9:137757794-137757994,constant,low_signal,LSC_high,low_signal
9:137814230-137814430,constant,low_signal,LSC_high,constant
9:137987361-137987561,constant,low_signal,constant,constant


In [12]:
# annotation_data
diff_annotations = ["LSC_high", "KC_high"]
variable_rows = (
    annotation_data.iloc[:, 0].isin(diff_annotations)
    | annotation_data.iloc[:, 1].isin(diff_annotations)
    | annotation_data.iloc[:, 2].isin(diff_annotations)
    | annotation_data.iloc[:, 3].isin(diff_annotations)
)
var_region_data = region_data.loc[variable_rows.values, :]
qnorm_intensities = var_region_data.loc[:, region_data.columns.str.contains("qnorm")]

#perform rowmean substraction per histone mod
for histmark in ["H3K27ac", "ATAC", "H3K4me3", "H3K27me3"]:
    col1 = f"KC_qnorm_{histmark}"
    col2 = f"LSC_qnorm_{histmark}"
    df = qnorm_intensities[[col1, col2]].astype(float)
    df = df.sub(df.mean(axis=1), axis=0)
    qnorm_intensities[col1] = df[col1]
    qnorm_intensities[col2] = df[col2]
    #perform motif enrichment with all diff/variable intensity regions to get the non-redudant motif db
    qnorm_intensities[[col1,col2]].to_csv(
        f"{MotifEnr.tmpdir}/{histmark}_int.txt",
        header=True,
        sep="\t",
        index=True,
    )
qnorm_intensities.to_csv(
    f"{MotifEnr.tmpdir}/all_marks_int.txt",
    header=True,
    sep="\t",
    index=True,
)


/scratch/jsmits/tmp/ipykernel_43191/553585019.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qnorm_intensities[col1] = df[col1]
/scratch/jsmits/tmp/ipykernel_43191/553585019.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qnorm_intensities[col2] = df[col2]


### Perform motif enrichment with all diff/variable intensity regions

In [ ]:
print(f"running maelstrom")

for TF_db in ["gimme.vertebrate.v5.0", "expressedTFs", "diffTFs"]:
    Path(f"{MotifEnr.output_dir}/maelstrom/{TF_db}/all_marks").mkdir(
        parents=True, exist_ok=True
    )
    if not os.path.exists(
        f"{MotifEnr.output_dir}/maelstrom/{TF_db}/all_marks/gimme.maelstrom.report.html"
    ):
        shutil.rmtree(
            "/ceph/rimlsfnwi/data/moldevbio/zhou/jsmits/.cache/gimmemotifs",
            ignore_errors=True,
        )
        shutil.rmtree("/home/jsmits/.config/gimmemotifs/", ignore_errors=True)
        print(f"running maelstrom for {TF_db}")
        sp.check_call(
            f"nice -15 gimme maelstrom "
            f"{MotifEnr.tmpdir}/all_marks_int.txt "
            f"{MotifEnr.files.T.genome[0]} "
            f"{MotifEnr.output_dir}/maelstrom/{TF_db}/all_marks "
            f"-p {MotifEnr.output_dir}/{TF_db}.pfm "
            f"--nthreads 4 "
            f"2> {MotifEnr.logdir}/maelstrom_{TF_db}_all_marks_log.txt",
            shell=True,
        )

running maelstrom
running maelstrom for gimme.vertebrate.v5.0
